In [2]:
import torch

# 4.1.2. Загрузка файла изображения
Изображения хранятся во многих файловых форматах, но, к счастью, в Python
существует множество способов загрузки изображений. Начнем с загрузки
изображения в формате PNG с помощью модуля imageio

In [3]:
import imageio

In [4]:
img_arr = imageio.v2.imread('data/bobby.jpg')

In [5]:
img_arr.shape

(720, 1280, 3)

Изображение здесь представляет собой подобный массиву NumPy объект с тремя
измерениями: два пространственных (ширина и высота) и третье измерение, соответствующее красному, зеленому и синему каналам. Для получения тензора
PyTorch подойдет любая библиотека, выдающая массивы NumPy. Следует только
учесть расстановку измерений. Модули PyTorch, работающие с изображениями,
требуют от тензоров схемы измерений C × H × W (каналы, высота и ширина
соответственно).

# 4.1.3. Изменение схемы расположения

Для получения нужной нам схемы расположения можно воспользоваться методом permute тензора, указав в качестве параметров старые измерения для каждого
из новых. При входном тензоре вида H × W × C, полученном нами ранее, нужную
схему расположения можно получить, указав сначала измерение 2, а затем 0 и 1:

In [6]:
img = torch.from_numpy(img_arr)
out = img.permute(2, 0, 1)

In [7]:
out

tensor([[[ 77,  77,  78,  ..., 118, 117, 116],
         [ 75,  76,  77,  ..., 118, 117, 116],
         [ 74,  75,  77,  ..., 117, 117, 116],
         ...,
         [215, 216, 217,  ..., 174, 176, 176],
         [215, 216, 217,  ..., 173, 174, 174],
         [215, 216, 217,  ..., 159, 158, 158]],

        [[ 45,  45,  46,  ...,  78,  77,  76],
         [ 43,  44,  45,  ...,  78,  77,  76],
         [ 39,  40,  43,  ...,  80,  78,  77],
         ...,
         [165, 166, 167,  ..., 121, 123, 123],
         [165, 166, 167,  ..., 123, 124, 124],
         [165, 166, 167,  ..., 109, 107, 107]],

        [[ 22,  22,  21,  ...,  52,  51,  50],
         [ 20,  21,  20,  ...,  52,  51,  50],
         [ 17,  18,  18,  ...,  51,  49,  48],
         ...,
         [ 78,  79,  80,  ...,  51,  53,  53],
         [ 78,  79,  80,  ...,  54,  55,  55],
         [ 78,  79,  80,  ...,  40,  41,  41]]], dtype=torch.uint8)

In [8]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

Как видим, наш батч будет состоять из трех RGB-изображений по 256 пикселей
высотой и 256 пикселей шириной. Обратите внимание на тип этого тензора:
мы предполагаем, что каждый из цветов будет представлен 8-битным целым
числом, как в большинстве форматов фотографий из стандартных бытовых
фотоаппаратов. Теперь можно загрузить все изображения в формате PNG из
входного каталога и сохранить их в тензоре:

In [9]:
import os

In [10]:
data_dir = 'data/image-cats/'
filenames = [name for name in os.listdir(data_dir) if os.path.splitext(name)[-1] == '.png']
# os.listdir - делает список файлов из поданного пути. 
# os.path.splitext - отделяет название и формат друг от друга. Получается кортеж (текст, формат) к примеру (cat, .png).

In [11]:
for index, filename in enumerate(filenames):
    img_arr = imageio.v2.imread(os.path.join(data_dir, filename))
    img = torch.from_numpy(img_arr)
    img_t = img.permute(2, 0, 1)
    img_t = img_t[:3] # Мы сохраняем только первые три канала. Иногда в изображениях встречается также альфа-канал, отвечающий за прозрачность, но нашей нейронной сети достаточно входных данных вида RGB
    batch[index] = img_t

# 4.1.4. Нормализация данных

Мы уже упоминали ранее, что роль входных сигналов нейронных сетей обычно играют тензоры с плавающей запятой. Нейронные сети демонстрируют
наилучшее качество обучения, когда входные данные находятся в диапазоне
примерно от 0 до 1 или от –1 до 1 (из-за того, как описаны их структурные
элементы).
Так что обычно требуется привести тип тензора к числам с плавающей запятой
и нормализовать значения пикселей. Приведение к числам с плавающей запятой
будет простой задачей, в то время как нормализация доставит много проблем,
поскольку зависит от выбираемого нами для приведения к интервалу от 0 до 1
(или от –1 до 1) диапазона входных данных. Один из вариантов: просто разделить
значения пикселей на 255 (максимальное число, которое можно представить
с помощью беззнакового 8-битного числа):

In [12]:
batch = batch.float()
batch = batch / 255.0

In [13]:
n_channels = batch.shape[1]
for c in range(n_channels):
    mean = torch.mean(batch[:, c])
    std = torch.std(batch[:, c])
    batch[:, c] = (batch[:, c] - mean) / std

# 4.2.1. Загрузка данных в специализированном формате

In [14]:
dir_path = 'data/2-LUNG 3.0  B70f-04083/'
vol_arr = imageio.v2.volread(dir_path, 'DICOM')
vol_arr.shape

Reading DICOM (examining files): 1/99 files (1.0%8/99 files (8.1%)

16/99 files (16.2%30/99 files (30.3%37/99 files (37.4%44/99 files (44.4%51/99 files (51.5%57/99 files (57.6%63/99 files (63.6%69/99 files (69.7%75/99 files (75.8%81/99 files (81.8%88/99 files (88.9%95/99 files (96.0%99/99 files (100.0%)
  Found 1 correct series.
Reading DICOM (loading data): 9/99  (9.117/99  (17.224/99  (24.232/99  (32.340/99  (40.448/99  (48.555/99  (55.661/99  (61.670/99  (70.778/99  (78.885/99  (85.992/99  (92.998/99  (99.099/99  (100.0%)


(99, 512, 512)

In [15]:
vol = torch.from_numpy(vol_arr).float()
vol = torch.unsqueeze(vol, 0)

In [16]:
vol.shape

torch.Size([1, 99, 512, 512])

# 4.3.2. Загрузка тензора данных по вину

In [17]:
import csv
import numpy as np

In [18]:
wine_path = 'data/winequality-white.csv'
wineq_numpy = np.loadtxt(wine_path, dtype=np.float32, delimiter=';',skiprows=1)

In [19]:
wineq_numpy

array([[ 7.  ,  0.27,  0.36, ...,  0.45,  8.8 ,  6.  ],
       [ 6.3 ,  0.3 ,  0.34, ...,  0.49,  9.5 ,  6.  ],
       [ 8.1 ,  0.28,  0.4 , ...,  0.44, 10.1 ,  6.  ],
       ...,
       [ 6.5 ,  0.24,  0.19, ...,  0.46,  9.4 ,  6.  ],
       [ 5.5 ,  0.29,  0.3 , ...,  0.38, 12.8 ,  7.  ],
       [ 6.  ,  0.21,  0.38, ...,  0.32, 11.8 ,  6.  ]], dtype=float32)

In [20]:
col_list = next(csv.reader(open(wine_path), delimiter=';'))

In [21]:
col_list

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [22]:
wineq_numpy.shape

(4898, 12)

In [23]:
wineq = torch.from_numpy(wineq_numpy)

# 4.3.3. Представление оценок

In [24]:
data = wineq[:,:-1] # Выбираем все строки и все столбцы, кроме последнего
data, data.shape

(tensor([[ 7.0000,  0.2700,  0.3600,  ...,  3.0000,  0.4500,  8.8000],
         [ 6.3000,  0.3000,  0.3400,  ...,  3.3000,  0.4900,  9.5000],
         [ 8.1000,  0.2800,  0.4000,  ...,  3.2600,  0.4400, 10.1000],
         ...,
         [ 6.5000,  0.2400,  0.1900,  ...,  2.9900,  0.4600,  9.4000],
         [ 5.5000,  0.2900,  0.3000,  ...,  3.3400,  0.3800, 12.8000],
         [ 6.0000,  0.2100,  0.3800,  ...,  3.2600,  0.3200, 11.8000]]),
 torch.Size([4898, 11]))

In [25]:
target = wineq[:,-1] # Выбираем все строки и последний столбец
target, target.shape

(tensor([6., 6., 6.,  ..., 6., 7., 6.]), torch.Size([4898]))

Если же преобразовывать тензор target в тензор меток, возможны два варианта
в зависимости от выбранной стратегии и того, для чего предназначаются категориальные данные. Первый заключается в том, что можно просто рассматривать
метки как целочисленный вектор оценок:

In [26]:
target = wineq[:, -1].long()
target

tensor([6, 6, 6,  ..., 6, 7, 6])

# 4.3.4. Быстрое кодирование

Для быстрого кодирования можно воспользоваться методом scatter_, заполняющим тензор значениями из исходного тензора по указанным в качестве
аргумента индексам:

In [27]:
target_onehot = torch.zeros(target.shape[0], 10)
target_onehot.scatter_(1, target.unsqueeze(1), 1.0)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

Аргументы метода scatter_ следующие:
 * измерение, по которому указываются следующие два аргумента;

 * тензор-столбец с индексами заполняемых элементов;
 
 * тензор с элементами для распределения или одно заполняющее скалярное
значение (в данном случае 1).

Размерность второго аргумента метода scatter_, тензора индексов, должна совпадать с заполняемым тензором. Поскольку размерность тензора target_onehot
равна 2 (4898 × 10), необходимо добавить еще одно фиктивное измерение в target
с помощью функции unsqueeze:

In [28]:
target_unsqueezed = target.unsqueeze(1)
target_unsqueezed

tensor([[6],
        [6],
        [6],
        ...,
        [6],
        [7],
        [6]])

В результате вызова unsqueeze добавляется одноэлементное измерение, превращая одномерный тензор в 4898 элементов в двумерный тензор размером (4898 × 1), без изменения его содержимого: никаких дополнительных
элементов не добавляется, так как мы просто решили использовать еще один
индекс для доступа к элементам. То есть к первому элементу target можно
обращаться target[0], а к первому элементу его аналога, к которому была применена unsqueeze, — target_unsqueezed[0,0].


# 4.3.5. Когда считать данные категориальными

Сначала вычислим
среднее значение и стандартное отклонение каждого из столбцов:

In [29]:
data_mean = torch.mean(data, dim=0)
data_var = torch.var(data, dim=0)

В данном случае dim=0 означает выполнение свертки по измерению 0. В этот
момент можно нормализовать данные посредством вычитания среднего значения
и деления на стандартное отклонение для улучшения процесса обучения

In [30]:
data_normalized = (data - data_mean) / torch.sqrt(data_var)

In [31]:
data_normalized

tensor([[ 1.7208e-01, -8.1761e-02,  2.1326e-01,  ..., -1.2468e+00,
         -3.4915e-01, -1.3930e+00],
        [-6.5743e-01,  2.1587e-01,  4.7996e-02,  ...,  7.3995e-01,
          1.3422e-03, -8.2419e-01],
        [ 1.4756e+00,  1.7450e-02,  5.4378e-01,  ...,  4.7505e-01,
         -4.3677e-01, -3.3663e-01],
        ...,
        [-4.2043e-01, -3.7940e-01, -1.1915e+00,  ..., -1.3130e+00,
         -2.6153e-01, -9.0545e-01],
        [-1.6054e+00,  1.1666e-01, -2.8253e-01,  ...,  1.0049e+00,
         -9.6251e-01,  1.8574e+00],
        [-1.0129e+00, -6.7703e-01,  3.7852e-01,  ...,  4.7505e-01,
         -1.4882e+00,  1.0448e+00]])

# 4.3.6. Поиск пороговых значений

In [32]:
bad_indexes = target <= 3 # PyTorch также предоставляет функции сравнения, здесь это torch.le(target, 3). Но использование операторов кажется хорошим вариантом
bad_indexes.shape, bad_indexes.dtype, bad_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(20))

In [33]:
bad_data = data[bad_indexes]
bad_data.shape

torch.Size([20, 11])

In [34]:
bad_data = data[target <= 3]
mid_data = data[(target > 3) & (target < 7)]
good_data = data[target >= 7]

bad_mean = torch.mean(bad_data, dim=0)
mid_mean = torch.mean(mid_data, dim=0)
good_mean = torch.mean(good_data, dim=0)

for i, args in enumerate(zip(col_list, bad_mean, mid_mean, good_mean)):
    print('{:2} {:20} {:6.2f} {:6.2f} {:6.2f}'.format(i, *args))

 0 fixed acidity          7.60   6.89   6.73
 1 volatile acidity       0.33   0.28   0.27
 2 citric acid            0.34   0.34   0.33
 3 residual sugar         6.39   6.71   5.26
 4 chlorides              0.05   0.05   0.04
 5 free sulfur dioxide   53.33  35.42  34.55
 6 total sulfur dioxide 170.60 141.83 125.25
 7 density                0.99   0.99   0.99
 8 pH                     3.19   3.18   3.22
 9 sulphates              0.47   0.49   0.50
10 alcohol               10.34  10.26  11.42


Давайте получим индексы, по которым столбец с общим содержанием сернистого ангидрида содержит
значение меньше среднего, вычисленного ранее, следующим образом:

In [35]:
total_sulfur_threshold = 141.83
total_sulfur_data = data[:, 6]
predicted_indexes = torch.lt(total_sulfur_data, total_sulfur_threshold)

predicted_indexes.shape, predicted_indexes.dtype, predicted_indexes.sum()

(torch.Size([4898]), torch.bool, tensor(2727))

In [36]:
actual_indixes = target > 5
actual_indixes.shape, actual_indixes.dtype, actual_indixes.sum()

(torch.Size([4898]), torch.bool, tensor(3258))

In [37]:
n_matches = torch.sum(actual_indixes & predicted_indexes).item()
n_predicted = torch.sum(predicted_indexes).item()
n_actual = torch.sum(actual_indixes).item()

n_matches, n_matches/n_predicted, n_matches/n_actual

(2018, 0.74000733406674, 0.6193984039287906)

# Тренировка знаний

### 1

In [38]:
batch_size = 3
batch = torch.zeros(batch_size, 3, 256, 256, dtype=torch.uint8)

In [39]:
filenames = [name for name in os.listdir('data/image-cats/') if os.path.splitext(name)[-1] == '.png']

for index, filename in enumerate(filenames):
    img = imageio.v2.imread(os.path.join('data/image-cats/', filename))
    img_t = torch.from_numpy(img)
    img_t = img_t.permute(2, 0, 1)
    img_t = img_t[:3]
    batch[index] = img_t

batch = batch / 255.0

### 2

In [40]:
path = 'data/train.csv'
data_train = np.loadtxt(path, dtype=np.float32, delimiter=',', skiprows=1)
data_t = torch.from_numpy(data_train)
col_list = next(csv.reader(open(path), delimiter=','))

In [41]:
col_list

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

# 4.4. Временные ряды

# 4.4.1. Добавляем измерение времени

In [42]:
path = 'data/bike/hour-fixed.csv'
bikes_numpy = np.loadtxt(path, 
                         dtype=np.float32,
                         delimiter=',', 
                         skiprows=1,
                         converters={1: lambda x: float(x[8:10])}) # Преобразует строки даты в числа, соответствующие дню месяца в столбце 1
bikes = torch.from_numpy(bikes_numpy)

In [43]:
bikes

tensor([[1.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 3.0000e+00, 1.3000e+01,
         1.6000e+01],
        [2.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 8.0000e+00, 3.2000e+01,
         4.0000e+01],
        [3.0000e+00, 1.0000e+00, 1.0000e+00,  ..., 5.0000e+00, 2.7000e+01,
         3.2000e+01],
        ...,
        [1.7377e+04, 3.1000e+01, 1.0000e+00,  ..., 7.0000e+00, 8.3000e+01,
         9.0000e+01],
        [1.7378e+04, 3.1000e+01, 1.0000e+00,  ..., 1.3000e+01, 4.8000e+01,
         6.1000e+01],
        [1.7379e+04, 3.1000e+01, 1.0000e+00,  ..., 1.2000e+01, 3.7000e+01,
         4.9000e+01]])

In [44]:
col_list = next(csv.reader(open(path), delimiter=','))

In [45]:
col_list

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

Чтобы получить нужный набор почасовых данных, нам достаточно просмотреть
тот же тензор батчами по 24 часа. Взглянем на форму и шаги тензора bikes:

In [46]:
bikes.shape, bikes.stride()

(torch.Size([17520, 17]), (17, 1))

Итого 17 520 часов и 17 столбцов. Изменим форму этих данных, чтобы получилось три оси координат: день, час, а затем наши 17 столбцов:

In [47]:
daily_bikes = bikes.view(-1, 24, bikes.shape[1])
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 24, 17]), (408, 17, 1))

Как вы помните также из предыдущей главы, хранилище — это непрерывный,
линейный контейнер для чисел (в данном случае с плавающей запятой). Каждая
строка нашего тензора bikes сохраняется вслед за предыдущей в соответствующем хранилище. Что подтверждается результатом приведенного выше вызова
bikes.stride().

Как видим, крайнее справа измерение содержит количество столбцов в исходном
наборе данных. Далее идет среднее измерение, которое содержит время, разбитое на порции по 24 последовательных часа. Другими словами, получается N
последовательностей по L часов в день для C каналов. Для получения желаемого
упорядочения N × C × L необходимо транспонировать тензор:

In [48]:
daily_bikes = daily_bikes.transpose(1, 2)
daily_bikes.shape, daily_bikes.stride()

(torch.Size([730, 17, 24]), (408, 1, 17))

Теперь применим к этому набору данных некоторые изученные ранее методики.

# 4.4.3. Готов для обучения

Чтобы упростить визуализацию данных, мы ограничимся пока что первым
днем. Заполним начальными значениями нулевую матрицу с числом строк,
равным количеству часов в сутках, и числом столбцов, равным числу уровней
погодных условий:

In [49]:
first_day = bikes[:24].long()
weather_onehot = torch.zeros(first_day.shape[0], 4)

In [50]:
first_day[:, 9]

tensor([1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2])

Теперь заполняем нашу матрицу единицами, в соответствии с уровнем погодных
условий в каждой строке. Не забудьте воспользоваться unsqueeze для добавления
одноэлементного измерения, как мы делали в предыдущем разделе

In [51]:
weather_onehot.scatter_(
    dim=1,
    index=first_day[:,9].unsqueeze(1).long() - 1, # Уменьшаем значения на 1, поскольку погодные условия варьируются от 1 до 4, в то время как индексы начинаются с 0
    value=1.0
)

tensor([[1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.]])

In [52]:
torch.cat((bikes[:24], weather_onehot), axis=1)[:1]

tensor([[ 1.0000,  1.0000,  1.0000,  0.0000,  1.0000,  0.0000,  0.0000,  6.0000,
          0.0000,  1.0000,  0.2400,  0.2879,  0.8100,  0.0000,  3.0000, 13.0000,
         16.0000,  1.0000,  0.0000,  0.0000,  0.0000]])

Можно было сделать то же самое с тензором измененной формы daily_bikes.
Помните, его форма (B, C, L), где L = 24. Сначала создаем нулевой тензор с теми
же B и L, но с числом дополнительных столбцов C

In [53]:
daily_weather_onehot = torch.zeros(daily_bikes.shape[0], 4, daily_bikes.shape[2])
daily_weather_onehot.shape

torch.Size([730, 4, 24])

Теперь заносим быстро кодированные значения в измерение C тензора. А поскольку эта операция выполняется там же, меняется только содержимое
тензора:

In [54]:
daily_weather_onehot.scatter_(
    dim=1,
    index=daily_bikes[:,9,:].long().unsqueeze(1) - 1, 
    value=1.0
)

daily_weather_onehot.shape

torch.Size([730, 4, 24])

In [55]:
daily_bikes = torch.cat((daily_bikes, daily_weather_onehot), dim=1)

Мы уже упоминали ранее, что это не единственный вариант, как можно поступить
с нашей величиной «погодные условия». И действительно, ее метки обладают
порядковой связью, так что можно притвориться, что это частные значения непрерывной величины. Просто преобразуем эту величину так, чтобы свести ее
область значений к промежутку от 0,0 до 1,0:

In [56]:
daily_bikes[:, 9, :] = (daily_bikes[:, 9, :] - 1.0) / 3.0

Как мы упоминали в предыдущем разделе, нормализацию данных к отрезку
[0,0, 1,0] или [–1,0, 1,0] желательно производить для всех количественных
величин, таких как temperature (столбец 10 в нашем наборе данных). Позднее
мы увидим, почему так, а пока скажем просто, что это полезно для процесса
обучения.

Существует множество вариантов нормализации величин. Можно просто отобразить их на диапазон [0,0, 1,0]:

In [57]:
temp = daily_bikes[:, 10, :]
temp_min = torch.min(temp)
temp_max = torch.max(temp)
daily_bikes[:, 10, :] = ((daily_bikes[:, 10, :] - temp_min) / (temp_max - temp_min))

или вычесть среднее значение и разделить на стандартное отклонение:

In [58]:
temp = daily_bikes[:, 10, :]
temp_mean = torch.mean(temp)
temp_std = torch.std(temp)
daily_bikes[:, 10, :] = (daily_bikes[:, 10, :] - temp_mean) / temp_std

# 4.5. Представление текста

Загрузим «Гордость и предубеждение» Джейн Остин с сайта проекта «Гутенберг»: www.gutenberg.org/files/1342/1342-0.txt. Просто сохраним файл и прочитаем
его в code/p1ch4/5_text_jane_austen.ipynb:

In [59]:
with open ('data/1342-0.txt', encoding='utf8') as f:
    text = f.read()

# 4.5.2. One-hot-кодирование символов

Сначала мы разобьем текст на список строк и выберем одну произвольную
строку, на которой сосредоточим свое внимание:

In [60]:
lines = text.split('\n')
line = lines[200]
line

'for there was a distinctly feminine element in “Mr. Spectator,” and in'

In [61]:
letter_t = torch.zeros(len(line), 128) # Число 128 жестко зашито в коде из-за ограничений ASCII
letter_t.shape

torch.Size([70, 128])

In [62]:
for i, letter in enumerate(line.lower().strip()):
    letter_index = ord(letter) if ord(letter) < 128 else 0 # В тексте используются двойные кавычки, которые не относятся к ASCII, так что мы их здесь отфильтруем
    letter_t[i][letter_index] = 1

In [63]:
letter_t[4]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])

# 4.5.3. Унитарное кодирование целых слов

In [78]:
def clean_words(input_str):
    punctuation = '.,;:"!?""_-“()#$&[]*”'
    word_list = input_str.lower().replace('\n', '').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)

word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

 0 5286 for
 1 13803 there
 2 15096 was
 3  141 a
 4 4044 distinctly
 5 5119 feminine
 6 4354 element
 7 7182 in
 8 9146 mr
 9 12751 spectator
10  821 and
11 7182 in


In [134]:
len(word_list)

16161

# Домашняя работа

1. Сделайте несколько снимков красных, синих и зеленых объектов с помощью
вашего телефона или любого цифрового фотоаппарата (или скачайте из
интернета, если фотоаппарата под рукой нет).

    А. Загрузите каждое из изображений и преобразуйте его в тензор.

    Б. Оцените яркость каждого из изображений с помощью метода .mean()
    соответствующего тензора.
    
    В. Вычислите среднее значение каждого из каналов ваших изображений.
    Можете ли вы распознать красные, синие и зеленые объекты по одному
    только среднему значению канала?

In [109]:
from pprint import pprint

In [119]:
path = 'data/homework/'
filenames = [name for name in os.listdir(path) if os.path.splitext(name)[-1] in ('.png', '.jpg')] 

info_img = dict()

for filename in filenames:
    img_numpy = imageio.v2.imread(os.path.join(path, filename))
    img_t = torch.from_numpy(img_numpy)
    img_t = img_t.permute(2, 0, 1).float()
    img_t/=255.
    mean_value = {'Red' : torch.mean(img_t[0]), 
                  'Green' : torch.mean(img_t[1]), 
                  'Blue' : torch.mean(img_t[2]), 
                  'Brightness' : torch.mean(img_t)}
    info_img[filename] = mean_value

In [120]:
pprint(info_img)

{'azer.png': {'Blue': tensor(0.3523),
              'Brightness': tensor(0.3766),
              'Green': tensor(0.3969),
              'Red': tensor(0.3808)},
 'eng.png': {'Blue': tensor(0.4935),
             'Brightness': tensor(0.6452),
             'Green': tensor(0.3862),
             'Red': tensor(0.7012)},
 'green.png': {'Blue': tensor(0.0118),
               'Brightness': tensor(0.6029),
               'Green': tensor(1.),
               'Red': tensor(0.4000)},
 'red.png': {'Blue': tensor(0.1419),
             'Brightness': tensor(0.5650),
             'Green': tensor(0.1580),
             'Red': tensor(0.9599)},
 'russia.jpg': {'Blue': tensor(0.6301),
                'Brightness': tensor(0.5651),
                'Green': tensor(0.4356),
                'Red': tensor(0.6296)},
 'white.jpg': {'Blue': tensor(0.9821),
               'Brightness': tensor(0.9821),
               'Green': tensor(0.9821),
               'Red': tensor(0.9821)}}


2. Выберите относительно большой файл с исходным кодом на Python.

    А. Создайте указатель всех слов в файле исходного кода (можете выбирать
    степень сложности токенизации по своему усмотрению; мы рекомендуем
    начать с замены r"[^a-zA-Z0-9_]+" пробелами).

    Б. Сравните свой указатель с созданным нами для «Гордости и предубеждения». Какой из них больше?

    В. Создайте унитарную кодировку для этого файла исходного кода.
    
    Г. Какая информация была утрачена при унитарномкодировании? Сравните
    с потерями при кодировании «Гордости и предубеждения»

In [121]:
with open ('data/homework/romeo.txt', encoding='UTF8') as file:
    text = file.read()

In [ ]:
def clean_words(input_str):
    punctuation = '.,;:"!?""_-“()#$&[]*”'
    word_list = input_str.lower().replace('\n', '').split()
    word_list = [word.strip(punctuation) for word in word_list]
    return word_list

words_in_line = clean_words(line)

word_list = sorted(set(clean_words(text)))
word2index_dict = {word: i for (i, word) in enumerate(word_list)}

word_t = torch.zeros(len(words_in_line), len(word2index_dict))
for i, word in enumerate(words_in_line):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1
    print('{:2} {:4} {}'.format(i, word_index, word))

In [139]:
def data_cleaning(input):
    words = input.lower().replace("\n",'').split()
    words = [word.strip('.,;:"!?""_-“()#$&[]*”') for word in words]
    return words
words = sorted(set(data_cleaning(text)))

In [140]:
word2index_dict = {word: i for (i, word) in enumerate(words)}

In [141]:
word_t = torch.zeros(len(words), len(word2index_dict))

In [143]:
for i, word in enumerate(words):
    word_index = word2index_dict[word]
    word_t[i][word_index] = 1